<a href="https://colab.research.google.com/github/zanuarts/dicoding-BPML/blob/master/dicoding_9_NLP_multclassTextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving imdb_indonesian_movies_2.csv to imdb_indonesian_movies_2.csv


In [ ]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns = ['judul_film'])

In [ ]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Melakukan proses one-hot-encoding karena dataset nya berupa kategorikal.

In [ ]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis = 1)
df_baru = df_baru.drop(columns = 'genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [ ]:
sinposis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Pembagian dataset menjadi Train dan Test

In [ ]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinposis, label, test_size = 0.2)

Tokenisasi

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 5000, oov_token = 'x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. 

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 5000, output_dim = 16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'softmax')
])
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
num_epochs = 30
history = model.fit(
    padded_latih,
    label_latih,
    epochs = num_epochs,
    validation_data = (padded_test, label_test),
    verbose = 2
)

Epoch 1/30
26/26 - 25s - loss: 1.6101 - accuracy: 0.2002 - val_loss: 1.6129 - val_accuracy: 0.1592
Epoch 2/30
26/26 - 28s - loss: 1.6047 - accuracy: 0.2127 - val_loss: 1.6166 - val_accuracy: 0.1791
Epoch 3/30
26/26 - 28s - loss: 1.6874 - accuracy: 0.3308 - val_loss: 1.6038 - val_accuracy: 0.2189
Epoch 4/30
26/26 - 28s - loss: 1.4574 - accuracy: 0.5597 - val_loss: 1.5593 - val_accuracy: 0.2687
Epoch 5/30
26/26 - 29s - loss: 1.0955 - accuracy: 0.5510 - val_loss: 1.8404 - val_accuracy: 0.2090
Epoch 6/30
26/26 - 28s - loss: 0.7517 - accuracy: 0.7251 - val_loss: 2.2002 - val_accuracy: 0.3383
Epoch 7/30
26/26 - 28s - loss: 0.4950 - accuracy: 0.8333 - val_loss: 2.2156 - val_accuracy: 0.3781
Epoch 8/30
26/26 - 28s - loss: 0.2903 - accuracy: 0.9092 - val_loss: 2.8969 - val_accuracy: 0.3483
Epoch 9/30
26/26 - 28s - loss: 0.1448 - accuracy: 0.9590 - val_loss: 3.2886 - val_accuracy: 0.3483
Epoch 10/30
26/26 - 28s - loss: 0.0651 - accuracy: 0.9863 - val_loss: 3.8782 - val_accuracy: 0.3781
Epoch 11/